In [1]:
# imports
import pickle
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

In [2]:
# load filtered df
# ???
# load final dataset
df = pd.read_csv('../raw_data/full_dataset.csv')

In [3]:
# # explore data
# df.describe()

In [4]:
# functionize seeding & feature selection
def get_seed_features(df):
    '''Get random seed track & features'''
    seed = df.sample()
    tempo = seed['scaled_tempo'].iat[0] 
    loudness = seed['scaled_loudness'].iat[0]
    da = seed['danceability'].iat[0]
    energy = seed['energy'].iat[0]
#     year = target['scaled_year'].iat[0]
#     popularity = target['popularity_binned'].iat[0]
    return tempo, loudness, da, energy #, popularity

In [5]:
# functionize trainig
def train_knn(df):
    '''Train the model'''
    features_names = ['scaled_tempo', 'scaled_loudness', 'danceability', 'energy'] # 'scaled_year', 'popularity_binned'
    X = df[features_names]
    y = df['track_id']
    model = KNeighborsRegressor(algorithm='kd_tree', n_jobs=-1).fit(X, y)
    return model

In [6]:
# train model
model = train_knn(df)
# pickle model
with open("../peak/knn_trained.pkl", "wb") as file:
    pickle.dump(model, file)   

In [7]:
# get seed features
tempo, loudness, da, energy = get_seed_features(df)

In [8]:
# load trained model
knn_trained = pickle.load(open("../peak/knn_trained.pkl","rb")) #file is in peak/peak

In [9]:
def get_k_dist_ind(knn_trained, tempo, loudness, da, energy):
    '''Get model output: distances & indices'''
    # set output & k
    knn_out, k = [], 100
    # get trained model output for k: distances & indices
    knn_out = knn_trained.kneighbors([[tempo, loudness, da, energy]], n_neighbors=k)
    return knn_out

In [11]:
# get output: distances & indices
knn_out = get_k_dist_ind(knn_trained,tempo, loudness, da, energy)
# get indices
ind = knn_out[1][0].tolist()

In [ ]:
# k=100 recommendations df
recs = df.iloc[ind] 
recs